In [7]:
#Keep the full length, but filter out invalid sequences. Run this before searching for promoter.
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re

i=0 #i is used to count the number of input sequences
j=0 #j is used to count the number of output sequences
valid_seq=[] #Create a blank list for those valid sequences
for rec in SeqIO.parse("../data/1_Original_fasta/MS2_only_-35_+5.fasta", "fasta",IUPAC.unambiguous_dna):
    i+=1 #Add 1 to total input number
    if re.search('\AATG[ATCG]+T[AG][GA]$', str(rec.seq[35:-5])): 
    # Check if the 51-53th are ATG, if all bases are ATCG and if there's a stop codon
        protein=rec.seq[35:-5].translate(to_stop=True) #Translate the putative CDS region to protein
        if len(protein)==(len(rec)-40)/3-1: #check if there's a internal stop codon
            rec_valid=SeqRecord(Seq(str(rec.seq),rec.seq.alphabet),id=rec.id,description='valid')
    # Save the original sequence, keep the original id number and add the 'valid' mark. 
            valid_seq.append(rec_valid) # Put the valid CDS to the list
            j+=1 #add one to the number of output sequences
SeqIO.write(valid_seq, "../data/2_valid_full_seq/valid_MS2_only.fasta", "fasta") 
# At last output the list of valid sequences as a new .fasta file
print ("Read %i sequences and write %i valid." %(i,j)) #Tell the user the numbers of input and output sequences 

Read 554 sequences and write 531 valid.


In [8]:
#This is to get the CDS of each gene, before calculating the CAI
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re

i=0 #i is used to count the number of input sequences
j=0 #j is used to count the number of output sequences
CDS_seq=[] #Create a blank list for those CDS sequences
for rec in SeqIO.parse("../data/1_Original_fasta/MS2_only_-35_+5.fasta", "fasta",IUPAC.unambiguous_dna): 
    i+=1 #Add 1 to total input number
    if re.search('\AATG[ATCG]+T[AG][GA]$', str(rec.seq[35:-5])): 
    # Check if the 51-53th are ATG, if all bases are ATCG and if there's a stop codon
        protein=rec.seq[35:-5].translate(to_stop=True) #Translate the putative CDS region to protein
        if len(protein)==(len(rec)-40)/3-1: #check if there's a internal stop codon
            rec_CDS=SeqRecord(Seq(str(rec.seq[35:-5]),rec.seq.alphabet),id=rec.id,description="Protein %i aa" %len(protein))
    # Save the coding sequence, keep the original id number and add the length of its protein product to the description. 
            CDS_seq.append(rec_CDS) # Put the valid CDS to the list
            j+=1 #add one to the number of output sequences
SeqIO.write(CDS_seq, "../data/3_CDS_fasta/CDS_MS2_only.fasta", "fasta") # At last output the list of valid sequences as a new .fasta file
print ("Read %i sequences and write %i CDS." %(i,j)) #Tell the user the numbers of input and output sequences 


Read 554 sequences and write 531 CDS.


In [3]:
#This script divides sequences into two groups based on the core transcriptional element type and trims from its TSS to the 100th nt in CDS.
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
import re

def Inr_m5_TSS_trimmer (file_name):
    records = [] #Create a new blank list for putative mRNA with Inr and m5 promoter
    i=0 #This is used to count the number of input sequences
    o_m5=0 #This is used to count the number of sequences with m5 element
    o_Inr=0 #This is used to count the number of sequences with Inr element
    for rec in SeqIO.parse(file_name, "fasta",IUPAC.unambiguous_dna): #Parse a .fasta file
        i+=1 #add one to the number of input sequences
        if re.search(r"CCTTT[ATCG]{0,15}ATG$",str(rec.seq[15:38])):
    #Check if there's an m5 element within the 20 nt upstream ATG start codon.
            m5_TSS=SeqRecord(Seq(re.search(r"C(CTTT[ATCG]{0,15}ATG[ATCG]{97})[ATCG]+$",\
                             str(rec.seq[15:-5])).group(1),rec.seq.alphabet),\
                             id=rec.id,description='m5_TSS_100CDS')
    #The second cytosine of ‘CCTTT’ in an m5 element is the TSS, Remove the 5' part.
            records.append(m5_TSS)
            o_m5+=1 #add one to the number of sequences with m5 element
        elif re.search(r"[TCA]CA[TCA][TA][ATCG]{0,25}ATG$",str(rec.seq[5:38])):
    #Check if there's an Inr element within the 30 nt upstream ATG start codon
            Inr_TSS=SeqRecord(Seq(re.search(r"[TCA]C(A[TCA][TA][ATCG]{0,25}ATG[ATCG]{97})[ATCG]+$",\
                              str(rec.seq[5:-5])).group(1),rec.seq.alphabet),\
                              id=rec.id,description='Inr_TSS_100CDS') 
    #The adenine of ‘TCANWY’ in an Inr element is the TSS. Remove the 5' part.
            records.append(Inr_TSS)
            o_Inr+=1 #add one to the number of sequences with Inr element
    SeqIO.write(records, "TSS_100CDS_"+file_name, "fasta") 
    # At last write the list of sequences with m5 or Inr element as new .fasta files
    print ("Read %i sequences in total. Among them %i have m5 element while %i have Inr element." %(i,o_m5,o_Inr)) 
    #Tell users the numbers of input and output sequences 

In [7]:
Inr_m5_TSS_trimmer('valid_MS2_only.fasta')

Read 531 sequences in total. Among them 35 have m5 element while 369 have Inr element.


In [3]:
#Randomize gene sequences with synonymous mutations
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from random import shuffle

output=[]
for rec in SeqIO.parse('test.fasta','fasta',IUPAC.unambiguous_dna):
    for i in range(100):
        UTR_5=str(rec.seq)[0:-100] #Slice the 5' UTR sequence
        UTR_list=list(UTR_5) #Convert the 5' UTR to a list 
        shuffle(UTR_list) #shuffle the elements in the list
        mut_UTR="".join(UTR_list) #Join the list back to a string
        CDS_99=str(rec.seq)[-100:-1] #Slice the CDS 
        CDS_codon_list=[CDS_99[n:n+3] for n in range(0, 99, 3)] #Convert the CDS to a list, each element being a codon
        shuffle(CDS_codon_list) #Shuffle the triplet codons
        mut_CDS_99="".join(CDS_codon_list) #Join the codon list back to a string
        Last=str(rec.seq)[-1] #Keep the last nt
        rec_Mut=SeqRecord(Seq(mut_UTR+mut_CDS_99+Last,rec.seq.alphabet),id=rec.id,description='Mut_%i'%(i))
        output.append(rec_Mut)
SeqIO.write(output,'mut_TSS_100CDS_m5_EST_and_MS.fasta','fasta')

300

In [4]:
#Find the sequences with top/bottom 200 translation efficiency
import pandas as pd
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re

i=0 #Count the number of sequences with m5 element
j=0 #Count the number of sequences with Inr element
output_top=[] #Store the first 100nt CDS of all top 200 sequences
output_bottom=[]#Store the first 100nt CDS of all bottom 200 sequences
effi_top200= pd.read_csv('../data/8_top_bottom_200_effi/effi_top_200.csv')
effi_top200_ID=list(effi_top200['Gene_ID'])
effi_bottom200= pd.read_csv('../data/8_top_bottom_200_effi/effi_bottom_200.csv')
effi_bottom200_ID=list(effi_bottom200['Gene_ID'])

for rec in SeqIO.parse('../data/5_TSS_100CDS_general_trend/TSS_100CDS_valid_EST5_and_MS2.fasta','fasta',IUPAC.unambiguous_dna):
    if rec.id in effi_top200_ID:
        output_top.append(rec)
        i+=1
    elif rec.id in effi_bottom200_ID: 
        output_bottom.append(rec)
        j+=1
SeqIO.write(output_top,'../data/8_top_bottom_200_effi/top200_TSS_100CDS.fasta','fasta')
SeqIO.write(output_bottom,'../data/8_top_bottom_200_effi/bottom200_TSS_100CDS.fasta','fasta')
print ('Obtained the TSS and 100 CDS of %i top200 and %i bottom200 sequences.' %(i,j))


Obtained the TSS and 100 CDS of 200 top200 and 200 bottom200 sequences.


In [1]:
#This is the script to produce a series of sliding windows with the width of 30 bp for each sequence
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import IUPAC

def window_30nt (file_name):
    window_records=[] #Create a new blank list for the windows of every sequence
    for rec in SeqIO.parse(file_name, "fasta"): #Parse a .fasta file
        for i in range(len(rec)-29): 
#Create a variable i keeping the index of each window, run the loop for n times based on the length of the sequence.
            index=101-len(rec)+i 
    #The index of each window is decided by its relative location to the start codon "ATG"
            window_rec=SeqRecord(Seq(str(rec.seq)[i:i+30], rec.seq.alphabet),\
                                 id="Window_%i_%s" %(index,rec.id),description="")
#Convert the sequence to a string, slice the string to 30 bp windows based on the index and add description to each window. 
            window_records.append(window_rec) #Add each window and its description to the list "window_records"
    SeqIO.write(window_records, "windows_"+file_name, "fasta") 
    # At last output the list of windows as a new .fasta file

In [2]:
window_30nt('nr_EST_only_TSS_100CDS.fasta')
window_30nt('nr_MS_only_TSS_100CDS.fasta')

In [3]:
#This is to isolate sequences of -4_+38 for each gene with both types of evidence
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import IUPAC

key_output=[] #Create a new blank list for the windows of every sequence
for rec in SeqIO.parse('../data/5_TSS_100CDS_general_trend_GC/TSS_100CDS_valid_EST5_and_MS2.fasta', "fasta"): 
    window_key=SeqRecord(Seq(str(rec.seq)[-104:-62], rec.seq.alphabet),\
                        id="Window_key_%s" %(rec.id),description="-4 to +38")
#Convert the sequence to a string, slice the string to 30 bp windows based on the index and add description to each window. 
    key_output.append(window_key) #Add each window and its description to the list "window_records"
SeqIO.write(key_output,'../data/5_TSS_100CDS_general_trend_GC/key_window_EST5_and_MS2.fasta', "fasta") 
    # At last output the list of windows as a new .fasta file

933

In [4]:
#This is to isolate the first 40 nt for each gene with both types of evidence
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import IUPAC

end_output=[] #Create a new blank list for the windows of every sequence
for rec in SeqIO.parse('../data/5_TSS_100CDS_general_trend_GC/TSS_100CDS_valid_EST5_and_MS2.fasta', "fasta"): 
    window_end=SeqRecord(Seq(str(rec.seq)[0:40], rec.seq.alphabet),\
                        id="5_end_%s" %(rec.id),description="First 40 nt from 5 terminus")
#Convert the sequence to a string, slice the string to 30 bp windows based on the index and add description to each window. 
    end_output.append(window_end) #Add each window and its description to the list "window_records"
SeqIO.write(end_output,'../data/5_TSS_100CDS_general_trend_GC/Five_terminus_EST5_and_MS2.fasta', "fasta") 
    # At last output the list of windows as a new .fasta file

933

In [3]:
#Extract MFE values of first 40nt or key window from RNAfold output
from Bio import SeqIO
import re
import numpy as np

Gene_ID=['Gene_ID']
MFE_5=['MFE_5terminus']
MFE_key=['MFE_key']
output=[]
for rec in SeqIO.parse("../data/5_TSS_100CDS_general_trend_GC/MFE_Five_terminus_EST5_and_MS2.fasta", "fasta"):
    Gene_ID.append(rec.id[6:])
    MFE_5.append(re.search(r"\(([\-0-9\.]+)\)$",str(rec.seq)).group(1))
for rec in SeqIO.parse("../data/5_TSS_100CDS_general_trend_GC/MFE_key_window_EST5_and_MS2.fasta", "fasta"):
    MFE_key.append(re.search(r"\(([\-0-9\.]+)\)$",str(rec.seq)).group(1))
output=np.vstack((Gene_ID,MFE_5,MFE_key))
np.savetxt('../data/5_TSS_100CDS_general_trend_GC/values_MFE_5terminus_key.csv',output.T,fmt='%s',delimiter=',')

In [2]:
#Extract MFE values for each 30nt wide window.
from Bio import SeqIO
import re
import io 

def MFE_value_extractor_plot (file_name):
    i=71 #This is a variable storing window numbers. It starts with the highest window number.
    out_MFE = [] #This is a blank list storing the final output
    while i>-30: #It is the condition when we continue the while loop.
        row_list=[] #This is a blank list storing a series of MFE values for a specific window
        row_list.append(str(i)) #This is the window index of each row
        for rec in SeqIO.parse(file_name, "fasta"): #Parse a .fasta file with MFE values
            if re.search(r"Window_([\-0-9]+)_",str(rec.id)).group(1) ==str(i): #Check if the current sequence comes from the same window 
                MFE_value=re.search(r"\(([\-0-9\.]+)\)$",str(rec.seq)).group(1) #Extract the MFE value
                row_list.append(str(MFE_value)) 
            #If yes, add the folding energy value to the end of the window index
        if len(row_list)>1: #If thereare more than one element (window index), continue.
            row_str=",".join(row_list) #After the for loop, join each element in the list to astring by ","
            out_MFE.append(row_str) #Add the string with MFEs separated by comma to the final output list
            i-=1 #Subtract one number to i and proceed the while loop to check the next window
        else: #If only a window index exists, jump out of the loop
             break
    with io.open ("values_"+file_name[0:-5]+"csv",'w',newline='') as f: #open a .csv file and write the output list
        f.write(unicode('\n'.join(out_MFE))) #Join the output list to a string by a carriage return. 
        #Must convert to unicode and use io.open in Python 2.7. Different in Python 3.6  
    print("values_"+file_name[0:-5]+"csv") #Tell users the names of the output .csv files

In [3]:
#MFE_value_extractor_plot('MFE_windows_nr_EST_only_TSS_100CDS.fasta')
MFE_value_extractor_plot('MFE_windows_nr_MS_only_TSS_100CDS.fasta')

values_MFE_windows_nr_MS_only_TSS_100CDS.csv


In [7]:
#This is to plot a series of boxplots showing MFE coverage given by synonymous iLOV genes
import numpy as np

final_output=[['window_index','Mean','STD','Count']]
with open('values_MFE_windows_mut_TSS_100CDS_EST5_or_MS2.csv','r') as f:    
    for line in f: #Read the csv file line by line
        line_output=[] #Save a series of window positions in this list variable
        values=[] #Save a series of lists in this list variable. Each list contains MFE values of a particular window.
        line_list=line.split(',') #Split each line to a list with ',' as a sep
        window_index=int(line_list[0]) #Save the window position in x_inde
        values=map(float,line_list[1:]) #Convert MFE values to float numbers
        values_arr=np.array(values)
        mean_value=np.mean(values_arr)
        std_value=np.std(values_arr)
        count_value=len(values)
        line_output=[window_index,mean_value,std_value,count_value]
        final_output.append(line_output)
np.savetxt('statistics_MFE_mut_EST5_or_MS2.txt',np.array(final_output),fmt='%s',delimiter='\t')

In [6]:
#This is to calculate the GC content values at each window position
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.SeqUtils import GC
import re
import io 

i=71 #This is a variable storing window numbers. It starts with the highest window number.
out_GC= [] #This is a blank list storing the final output
while i>-30: #It is the condition when we continue the while loop.
    row_list=[str(i)] #This is a blank list storing a series of GC content values for a specific window
    for rec in SeqIO.parse('../../permutation_mass_data/windows_mut_EST5_or_MS2.fasta',\
                           'fasta',IUPAC.unambiguous_dna): #Parse a .fasta file with MFE values
        if re.search(r"Window_([\-0-9]+)_",str(rec.id)).group(1) ==str(i): #Check if the current sequence comes from the same window 
            GC_content=round(GC(rec.seq),2) #Extract the MFE value
            row_list.append(str(GC_content))
            #If yes, add the folding energy value to the end of the window index
    if len(row_list)>1: #If thereare more than one element (window index), continue.
        row_str=",".join(row_list) #After the for loop, join each element in the list to astring by ","
        out_GC.append(row_str) #Add the string with MFEs separated by comma to the final output list
        i-=1 #Subtract one number to i and proceed the while loop to check the next window
    else: #If only a window index exists, jump out of the loop
        break
with io.open ('../data/6_TSS_100CDS_permutation/values_GC_mut_EST5_or_MS2.csv','w',newline='') as f: #open a .csv file and write the output list
    f.write(unicode('\n'.join(out_GC))) #Join the output list to a string by a carriage return. 
        #Must convert to unicode and use io.open in Python 2.7. Different in Python 3.6  

In [4]:
#This is to calculate the mean value and STD of GC contents for each window
import pandas as pd
from pandas import DataFrame

def GC_statistics (file_name):
    GC_content = pd.read_csv(file_name,index_col=0,header=None)
#Read a .csv file. "index_col=0" makes the first column the index for each row. "header=None" says there's no colomn index.
    column1=DataFrame(GC_content.mean(axis=1), columns=['Mean'])
#Calculate the mean value of each row, add index "Mean" on top of this column and save it to a variable as a DataFrame
    column2=DataFrame(GC_content.std(axis=1), columns=['STD'])
#Calculate the STD of each row, add index "STD" on top of this column and save it to a variable as a DataFrame
    column3=DataFrame(GC_content.count(axis=1), columns=['Count'])
#Count the number of values of each row    
    columns=pd.concat([column1, column2,column3], axis=1)
    #Concatenate the above two columns together
    columns.to_csv('statistaics_'+file_name)
    #write the result as a new .csv file.

In [8]:
#This is to calculate the mean and STD of GC contents for permutated sequences
import numpy as np

final_output=[['window_index','Mean','STD','Count']]
with open('../data/6_TSS_100CDS_permutation/values_GC_mut_EST5_and_MS2.csv','r') as f:    
    for line in f: #Read the csv file line by line
        line_output=[] #Save a series of window positions in this list variable
        values=[] #Save a series of lists in this list variable. Each list contains MFE values of a particular window.
        line_list=line.split(',') #Split each line to a list with ',' as a sep
        window_index=int(line_list[0]) #Save the window position in x_inde
        values=map(float,line_list[1:]) #Convert MFE values to float numbers
        values_arr=np.array(values)
        mean_value=np.mean(values_arr)
        std_value=np.std(values_arr)
        count_value=len(values)
        line_output=[window_index,mean_value,std_value,count_value]
        final_output.append(line_output)
np.savetxt('../data/6_TSS_100CDS_permutation/statistics_GC_mut_EST5_and_MS2.txt',np.array(final_output),fmt='%s',delimiter='\t')